In [ ]:

import kagglehub
avikumart_movies_reviews_dataset_path = kagglehub.dataset_download('avikumart/movies-reviews-dataset')

print('Data source import complete.')


# Introduction

**In today's world, one of the biggest sources of information is text data, which is unstructured in nature, finding customer sentiments from product reviews or feedback, extracting opinions from social media data are a few examples of text analytics.**

**Finding Insights from text data is not as straight forward as structured data and it need extensive data pre-precessing. pre-precessing inclues removing punctuations, removing stop-words and cleaning words to it root format so that vectorized data contains meaningful features that maps target variable well enough.**

**In this notebook, we will use `Count vectorizer` & `TF-IDF vectorizer` along with stemming and n-grams to pre-precess and vectorize the movie reviews data before we build sentiment classification models for training and testing.**

![image.png](attachment:f3dbf741-e35d-4e3d-95e9-a82a6b98ef72.png)

# Importing libraries

In [ ]:
# standard data science library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

# Text processing and model building libraries
import random
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from nltk.stem.snowball import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

# matplotlib defaults
plt.style.use("seaborn-darkgrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load and explore a dataset

In [ ]:
train_ds = pd.read_csv("/kaggle/input/movies-reviews-dataset/sentiment_train", delimiter="\t")
train_ds.head(10)

In [ ]:
train_ds.shape

In [ ]:
train_ds.info()

In [ ]:
train_ds.describe(include=['object','int64'])

**Insight:** In our dataset there are 1291 unique reviews and most frquent one is 'I love Harry Potter'.

In [ ]:
# set width of the columns as 800 charactors
pd.set_option('max_colwidth', 800)
train_ds[train_ds.sentiment == 1][:5] # five positive sentiments

In [ ]:
# five negative sentiments
train_ds[train_ds.sentiment == 0][:5]

In [ ]:
#  number of reviews per sentiment
sen_df = train_ds['sentiment'].value_counts().reset_index()
sen_df['% of reviews'] = sen_df['sentiment'].map(lambda x : (x/sen_df['sentiment'].sum())*100)
sen_df

**Insight:**

**Approx 60% of reviews are positive while 43% of them are negative ones.**

In [ ]:
# plot the sentiments using count plot of the seaborn
plt.figure(figsize=(6,6))
ax = sns.countplot(x='sentiment', data=train_ds)
for p in ax.patches:
    ax.annotate(p.get_height(), (p.get_x()+0.1, p.get_height() + 50))
plt.title("Positive and negative sentiments")
plt.xlabel("Sentiment")
plt.ylabel("Count of sentiment")
plt.show()

# Text Pre-processing and vectorization of text data

**BOW (Bag of words model):**

1. Count Vector model
2. Term Frequecny Vector model
3. Term-Frequency-Inverse Document Frequency (TF-IDF) Model

In out dataset, each text line will be called as a document, which will be used as vector to calculate count vecotrizer and TF/TF-IDF vectors.

# 1. Using Count vectorizer

In [ ]:
# Initialize the count vectorizer
count_vectorizer = CountVectorizer()
# Create the dictionary from the corpus
feature_vector = count_vectorizer.fit(train_ds.text)
# get the feature names
features0 = feature_vector.get_feature_names()
print("Total number of features: ", len(features0))
print("-----------------------------------------")
# name of the each features
print(feature_vector.get_feature_names_out())
print("-----------------------------------------")
# random feature names
print(random.sample(features0, 10))

### Transform features to sparse matrix

In [ ]:
# tranform the data frame to a vectorizer
train_ds_features = count_vectorizer.transform(train_ds.text)
print(type(train_ds_features))
print("-----------------------------------------")
# shape of transoformed array
print(train_ds_features.shape)
print("-----------------------------------------")
# how sparse is this out matrix, find the non-zero values in matrix
print("Non-zero values in our matrix:", train_ds_features.getnnz())
print("-----------------------------------------")
# percentage of total values set as a zeros
print("Density of the matrix: ", train_ds_features.getnnz()*100/(train_ds_features.shape[0]*train_ds_features.shape[1]))

### Convert sparse matrix to dense pandas dataframe

In [ ]:
# converting to dense dataframe
train_ds_df = pd.DataFrame(train_ds_features.todense())
train_ds_df.columns = features0
# let's croos-check with original dataframe with our vectorised dataframe
train_ds[0:1]

In [ ]:
# first documents 150 to 157 features has 'awesome' word as a feature and it can be seen counted as 1.
train_ds_df.iloc[:1, 150:157]

In [ ]:
# let's also check words in our first document as its features in our dense dataframe
train_ds_df[['the','da','vinci','code','book','is','just','awesome','something']][:1]

In [ ]:
# let''s find out which words are most occuring
feature_counts = train_ds_df.sum().reset_index().sort_values(by=0, ascending=False)
feature_counts.columns = ['Features', 'Counts']
feature_counts.head(10)

In [ ]:
# plot the distribution of words count
plt.figure(figsize=(12,5))
plt.hist(feature_counts.Counts, bins=50, range= (0,2000))
plt.xlabel("Frequecny of the words")
plt.ylabel("Density")
plt.title("Distribution of frequecny of each words in corpus")
plt.show()

In [ ]:
# find the rare words, words that are appear only 1 time
len(feature_counts[feature_counts.Counts == 1])

**Insight:**

**Out of total 2132 words/features 1228 appears only 1 time, we can remove non-significant words**

### Remove-Stop Words

In [ ]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(['harry','potter','code','vinci','da','harry','mountain',
                                              'movie','movies'])
my_stop_words1 = list(my_stop_words)

In [ ]:
# setting stop-wrod list in countvectorizer
count_vectorizer1 = CountVectorizer(stop_words = my_stop_words1,
                                   max_features = 1000)    #keeping only 1000 top words
feature_vector1 = count_vectorizer1.fit(train_ds.text)
train_ds_features1 = count_vectorizer1.transform(train_ds.text)
features = feature_vector1.get_feature_names()
features_counts1 = np.sum(train_ds_features1.toarray(), axis=0)
feature_counts1 = pd.DataFrame(dict(features = features,
                                    counts = features_counts1))
feature_counts1.sort_values('counts', ascending=False)[:15]

### Stemming and lemmatization

**Stemming: It chops-off the words to the stemmed words**

**LemmatizationL It converts the words into its root form (i.e English Dictionary)**

![image.png](attachment:d76b1387-d2b5-44c6-a6e7-7404a4fad990.png)

In [ ]:
# let's learn what is stemming and lemmatization by example
print("Original:")
print(train_ds.text[1])
print()

sentence = []
for word in train_ds.text[1].split():
    stemmer = PorterStemmer()
    sentence.append(stemmer.stem(word))
print("Stemming:")
print(' '.join(sentence))
print()

sentence = []
for word in train_ds.text[1].split():
    lemmatizer = WordNetLemmatizer()
    sentence.append(lemmatizer.lemmatize(word))
print("Lemmatizing")
print(' '.join(sentence))
print()

In [ ]:
# text pre-processing with stop-words and stemming
analyzer = CountVectorizer().build_analyzer()
# custom function for stemming and stop word reomval
def stemming_words(doc):
    stemmed_words = [stemmer.stem(w) for w in analyzer(doc)] # stemming the words
    non_stop_words = [word for word in stemmed_words if word not in my_stop_words1] # keep non stop words only
    return non_stop_words

In [ ]:
count_vectorizer2 = CountVectorizer(analyzer=stemming_words, max_features=1000)
feature_vector2 = count_vectorizer2.fit(train_ds.text)
train_ds_features2 = count_vectorizer2.transform(train_ds.text)
features = feature_vector2.get_feature_names()
features_counts2 = np.sum(train_ds_features2.toarray(), axis=0)
feature_counts2 = pd.DataFrame(dict(features = features,
                                    counts = features_counts2))
feature_counts2.sort_values('counts', ascending=False)[:15]

In [ ]:
# Crete final train data frame to build models
trn_ds_df = pd.DataFrame(train_ds_features2.todense()) # convert sparse array to dataframe
trn_ds_df.columns = features  # assign features of train daraframe
trn_ds_df['sentiment'] = train_ds.sentiment
trn_ds_df.head()

In [ ]:
trn_ds_df.shape

In [ ]:
bar_df = feature_counts2.sort_values('counts', ascending=False)[:15]
feature_list = list(bar_df.features)

fig, ax = plt.subplots(5,3, figsize=(12,15))
for i, j in enumerate(feature_list):
    sns.barplot(x='sentiment', y=j ,data=trn_ds_df, estimator=sum, ax=ax[i//3, i%3])
    ax[i//3, i%3].grid(visible=True, linestyle='--')
    ax[i//3, i%3].set_title(f'for feature/word {j}')

plt.suptitle("Sentiments of top most used words in reviews", fontsize=17, fontweight='bold')
plt.show()

**Insight:**

**We can see word/feature appreance and its sentiment mostly negative words like 'hate', 'stupid' are having (0) negative sentiment and positive words 'love', 'like' are having positive sentiment.**

a**lso by keeping most used words which most likely to map target variable will improve sentiment classification accuracy.**

### Wordcloud of reviews

In [ ]:
plt.figure(figsize=(15,15))
wc = WordCloud(max_words=1000,
               min_font_size=10,
               height=600,
               width=1600,
               background_color='white',
               stopwords=STOPWORDS).generate(' '.join(train_ds.text))

plt.imshow(wc, interpolation= "bilinear")
plt.axis('off')

### Naive-Bayes model for sentiment classification

In [ ]:
train_X,test_X,train_y,test_y = train_test_split(train_ds_features2, train_ds.sentiment,
                                                   test_size = 0.3, random_state=42)
print(train_X.shape)
print(test_X.shape)

# naive-bayes classifier
nb_clf = BernoulliNB()
nb_clf.fit(train_X.toarray(), train_y)
prediction = nb_clf.predict(test_X.toarray())

In [ ]:
# classification report of test out output and prediction
classificationreport = classification_report(test_y, prediction)
print(classificationreport)

In [ ]:
# plot the confusion matrix
cm = confusion_matrix(test_y, prediction)
sns.heatmap(cm, annot=True, fmt=".2f")
plt.title("Confusion Matrix of NB classifier")
plt.ylabel("Actual values")
plt.xlabel("Predicted values")
plt.show()

**Observation:**

**Naive-Bayes model gives you an accuracy of 98% for sentiment classification model.**

# 2. Using TF-IDF vectorizer

In [ ]:
trn_X,tst_X,trn_y,tst_y = train_test_split(train_ds.text, train_ds.sentiment,
                                                   test_size = 0.3, random_state=42)

# transform dataset into cleaned vectorizors
tfidf_vectorizer = TfidfVectorizer(analyzer= stemming_words,
                                   max_features = 1000)
feature_train = tfidf_vectorizer.fit_transform(trn_X)
feature_test = tfidf_vectorizer.transform(tst_X)

In [ ]:
# models to be trained
models = [DecisionTreeClassifier(),
          RandomForestClassifier(),
          LogisticRegression(max_iter=1000),
          KNeighborsClassifier(),
          BernoulliNB()]

In [ ]:
accuracy = []

for model in models:
    cross_val = cross_val_score(model, feature_train, trn_y, scoring='accuracy',
                                cv= StratifiedKFold(10)).mean()
    accuracy.append(cross_val)

In [ ]:
models_name = ['DecisionTreeClassifier', 'RandomForestClassifier',
         'LogisticRegression', 'KNeighborsClassifier', 'BernoulliNB']

accuracy_df = pd.DataFrame({'Model': models_name, 'Accuracy': accuracy})
accuracy_df

**Logistic Regression is best performer for our dataset so will build loistic regression model for sentiment classification.**

In [ ]:
logistic_reg = LogisticRegression(max_iter=1000)
logistic_reg.fit(feature_train, trn_y)
prediction_log = logistic_reg.predict(feature_test)
clf_report = classification_report(tst_y, prediction_log)
print(clf_report)

In [ ]:
# plot the confusion matrix
cm_log = confusion_matrix(tst_y, prediction_log)
sns.heatmap(cm_log, annot=True, fmt=".2f")
plt.title("Confusion Matrix of logistics regression classifier")
plt.ylabel("Actual values")
plt.xlabel("Predicted values")
plt.show()

**Observation:**

**With logistic regression we got around 99% accuracy which is more than Naive-Bayes classifier apparently.**

# 3. Using n-Grams range with TF-IDF vectorizor

In [ ]:
# define get_stemmed_tokens for n-grams
def get_stemmed_tokens(doc):
    all_tokens = [word for word in nltk.word_tokenize(doc)]
    clean_tokens = []
    for each_token in all_tokens:
        if re.search('[a-zA-Z]', each_token):
            clean_tokens.append(each_token)
    # stem the words
    stemmed_tokens = [stemmer.stem(t) for t in clean_tokens]
    return stemmed_tokens

In [ ]:
tfidf_vector = TfidfVectorizer(max_features=1000,
                               stop_words='english',
                               tokenizer = get_stemmed_tokens,
                               ngram_range=(1,2))

feature_trn = tfidf_vector.fit_transform(trn_X)
feature_tst = tfidf_vector.transform(tst_X)
# build Naive-bayes model
NB_model = BernoulliNB()
NB_model.fit(feature_trn, trn_y)
preds = NB_model.predict(feature_tst)
clf_report = classification_report(tst_y, preds)
print(clf_report)

**Observation:**

**Using n-Grams range of (1,2) we received 98% accuracy of sentiment classification using Naive-Bayes model.**

# Conclusion:

**1. We build 5 different models using Count vectorization and TF-IDF vectorization techniques to extract features from text data**

**2. We used stop-words removal, stemming and regular expressions for text data cleaning**

**3. With Logistic regression model we recieved highest accuracy of 99% and with Naive-Bayes model we recieved accuracy of 98%.**